In [8]:
import os

import random
import numpy as np
import pandas as pd
import mne
from sklearn.model_selection import train_test_split
from settings import EEGSettings
from eeg_lib import  create_dataset, load_data, predict_lm, predict_lgbm, plot_patterns, plot_clusters, plot_roc_curves, plot_feature_importance

In [2]:
# path to data
path = '/Users/ilyamikheev/Documents/IHNA/eeg_data'

# create settings object and initialise montage
settings = EEGSettings(path=path)
montage = mne.channels.make_standard_montage(settings.montage_name)

# Subject indexes in both groups
mat_indexes = settings.mat_indexes
not_mat_indexes = settings.not_mat_indexes
all_indexes = settings.order_indexes

new_mat = random.sample(mat_indexes , 10)
new_not_mat = random.sample(not_mat_indexes, 10)

<center><b>Create features space</b></center>

In [ ]:
# create dataset and save on disc
data = create_dataset(settings, montage)

In [3]:
# load from disc
data = load_data(['raw_epochs', 'spectra_feat', 'mean_spectra', 'chan_names', 'info_object'])

Loading of files: 100%|██████████| 5/5 [00:19<00:00,  3.94s/it]


<center><b>Classification of tasks in both groups</b></center>

In [10]:
# linear regression
results = np.zeros((len(all_indexes), len(settings.dict_cls), 2))
coefs =  np.zeros((len(all_indexes), len(settings.dict_cls), 2, len(settings.fr_bands), len(data['chan_names'])))
tprs, aucs, true_v, pr_v = [], [], [], []

for subj in range(len(all_indexes)):
    ev_tpr, ev_auc, ev_pred, ev_true = [], [], [], []
    for i, key in enumerate(list(settings.dict_cls.keys())):
        ind = settings.dict_cls[key]
        A, B = data['spectra_feat'][subj][ind[0]], data['spectra_feat'][subj][ind[0]]
        x = np.concatenate((A,B), axis=0)
        y = np.array([0]*A.shape[0] + [1]*B.shape[0])
        x_train, x_test, y_train, y_test = train_test_split(x.reshape(x.shape[0],-1), y, test_size=0.3)
        ac, roc_auc, interp_tpr, coef_ev, y_predict_pr = predict_lm([x_train, x_test, y_train, y_test], [settings.fr_bands,                                                                                                        data['chan_names']])
        results[subj, i, ...] = ac, roc_auc
        coefs[subj, i, ...] = coef_ev
        ev_pred.append(y_predict_pr)
        ev_true.append(y_test)
        ev_tpr.append(interp_tpr)
        ev_auc.append(roc_auc)
    tprs.append(ev_tpr)
    aucs.append(ev_auc)
    true_v.append(ev_true)
    pr_v.append(ev_pred)

TypeError: 'list' object cannot be interpreted as an integer

In [ ]:
# evaluate mean and var for all subjects/and for subjects in groups
list_gr, ar_mean_var = [results, results[index_mat,...], results[index_not_mat,...]], np.zeros((3,12))
list_names_subj, list_names_fin = [], []
metr_name = ['accuracy','ROC/AUC']
for i, key in enumerate(list(dict_cls.keys())):
    ar_mean_var[i,...] = np.array(list(chain.from_iterable((a, b) for a,b in zip(list_gr[i].mean(axis=0),
                                                                                 list_gr[i].var(axis=0)))))
    list_names_subj.extend(['{} {}'.format(key,name) for name in metr_name])
    list_names_fin.extend(['{} {}'.format(key,name) for name in ['{} {}'.format(i,j) for i in metr_name
                                                                 for j in ['mean','var'] ]])

In [ ]:
# save results to table
df_subj = pd.DataFrame(results, columns=list_names_subj, index=indexes)
df_fin = pd.DataFrame(ar_mean_var, columns=list_names_fin,
                       index=['all_subjects','mathematicians','not_mathematicians'])
writer = pd.ExcelWriter(os.path.join('/Users/ilyamikheev/Downloads/', 'subjects_classification_tasks_linear.xlsx'), engine='xlsxwriter')
df_subj.to_excel(writer, sheet_name='all_subj')
df_fin.to_excel(writer, sheet_name='mean_var')
writer.save()

In [ ]:
import matplotlib.pyplot as plt

from sklearn import svm
from sklearn.metrics import auc
from sklearn.metrics import RocCurveDisplay

random_state = np.random.RandomState(0)

#tprs = tprs_f[2]
tprs  = [tprs_f[2][i] for i in index_not_mat]
#aucs = aucs_f[2]
aucs = [aucs_f[2][i] for i in index_not_mat]
fig, ax = plt.subplots(figsize=(12, 8))

ax.plot([0, 1], [0, 1], linestyle="--", lw=2, color="r", label="Chance", alpha=0.8)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
ax.plot(
    mean_fpr,
    mean_tpr,
    color="b",
    label=r"Mean ROC (AUC = %0.2f $\pm$ %0.2f)" % (mean_auc, std_auc),
    lw=2,
    alpha=0.8,
)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
ax.fill_between(
    mean_fpr,
    tprs_lower,
    tprs_upper,
    color="grey",
    alpha=0.2,
    label=r"$\pm$ 1 std. dev.",
)

ax.set(
    xlim=[-0.05, 1.05],
    ylim=[-0.05, 1.05],
    title="Receiver operating characteristic Logistic reression",
)
ax.legend(loc="lower right")
plt.show()

In [ ]:
a_lf = []
s = coefs[0][0].shape
a_l = coefs[2][index_not_mat]
for i in range(len(index_not_mat)):
    k = minmax_scale(a_l[i].reshape(2,-1), feature_range=(-1,1), axis=1)
    a_lf.append(k.reshape(s))
fin_ar = np.mean(a_lf, axis=0)
fig = plot_patterns(fin_ar)

In [ ]:
fig.savefig(os.path.join('/Users/ilyamikheev/Downloads/', 'feature_patterns_241_242_not_mat.png'), format='png', bbox_inches='tight', dpi=600)

In [ ]:
fig.savefig(os.path.join('/Users/ilyamikheev/Downloads/', 'lr_roc_auc_241_242_not_mat.png'), format='png', bbox_inches='tight', dpi=600)

In [ ]:
# classification for each subject in both groups lgb
mean_fpr = np.linspace(0, 1, 100)
tprs_f, aucs_f = [], []
fpr_f, tpr_f = [], []
ind = 0
im = []
results = np.zeros((len(indexes),6))
k = 2
for i, key in enumerate(list(dict_cls.keys())):
    im1 = []
    tprs = []
    aucs = []
    fprl = []
    tprl = []
    # change to zip
    for subj in range(len(indexes)):
        ind = dict_cls[key]
        A, B = np.stack(subj_list_features[subj][ind[0]], axis=1), np.stack(subj_list_features[subj][ind[1]], axis=1)
        y = np.array([0] * A.shape[0] + [1] * B.shape[0])
        x = np.concatenate((A, B), axis=0)
        x_train, x_test, y_train, y_test = train_test_split(x.reshape(x.shape[0], -1), y, test_size=0.3)

        results[subj,k-2:k] = (ac, auc)
        fpr, tpr, _ = roc_curve(y_test, predictions[:,1])
        fprl.append(predictions)
        tprl.append(y_test)
        roc_auc = metrics.auc(fpr, tpr)
        interp_tpr = np.interp(mean_fpr, fpr, tpr)
        interp_tpr[0] = 0.0
        tprs.append(interp_tpr)
        aucs.append(roc_auc)
    k+=2
    im.append(im1)
    tpr_f.append(tprl)
    fpr_f.append(fprl)
    tprs_f.append(tprs)
    aucs_f.append(aucs)

In [ ]:
# evaluate mean and var for all subjects/and for subjects in groups
list_gr, ar_mean_var = [results,results[index_mat,...],results[index_not_mat,...]], np.zeros((3,12))
list_names_subj, list_names_fin = [], []
metr_name = ['accuracy','ROC/AUC']
for i, key in enumerate(list(dict_cls.keys())):
    ar_mean_var[i,...] = np.array(list(chain.from_iterable((a, b) for a,b in zip(list_gr[i].mean(axis=0),
                                                                                 list_gr[i].var(axis=0)))))
    list_names_subj.extend(['{} {}'.format(key,name) for name in metr_name])
    list_names_fin.extend(['{} {}'.format(key,name) for name in ['{} {}'.format(i,j) for i in metr_name
                                                                 for j in ['mean','var'] ]])

In [ ]:
# save results to table
df_subj = pd.DataFrame(results, columns=list_names_subj, index=indexes)
df_fin = pd.DataFrame(ar_mean_var, columns=list_names_fin,
                       index=['all_subjects','mathematicians','not_mathematicians'])
writer = pd.ExcelWriter(os.path.join('/Users/ilyamikheev/Downloads/', 'subjects_classification_tasks_lgb.xlsx'), engine='xlsxwriter')
df_subj.to_excel(writer, sheet_name='all_subj')
df_fin.to_excel(writer, sheet_name='mean_var')
writer.save()

In [ ]:
#tprs = tprs_f[2]
tprs  = [tprs_f[2][i] for i in index_mat]
#aucs = aucs_f[2]
aucs = [aucs_f[2][i] for i in index_mat]
fig, ax = plt.subplots(figsize=(12, 8))

ax.plot([0, 1], [0, 1], linestyle="--", lw=2, color="r", label="Chance", alpha=0.8)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)


In [ ]:
fig.savefig(os.path.join('/Users/ilyamikheev/Downloads/', 'lgb_roc_auc_241_242_math_subj.png'), format='png', bbox_inches='tight', dpi=600)

In [ ]:
#a_lf = im[2]
a_lf = [im[2][i] for i in index_not_mat]
fin_ar = np.mean(a_lf, axis=0).reshape((7,95))
fig = plot_topo(fin_ar, info1)

In [ ]:
fig.savefig(os.path.join('/Users/ilyamikheev/Downloads/', 'feature_imp_241_242_not_mat.png'), format='png', bbox_inches='tight', dpi=600)

In [ ]:
new_mat = ['326', '330', '316', '328', '314', '335', '327', '312', '317', '334']
new_not_mat = ['333', '320', '319', '336', '331', '323', '313', '324', '325', '337']
index_mat, index_not_mat = [indexes.index(i) for i in new_mat], [indexes.index(i) for i in new_not_mat]

In [ ]:
# group linear model
import random
list_A, list_B = [], []
new_mat = ['311', '316', '312', '334', '313', '332', '324', '337']
new_not_mat = ['314','327','326','335','329','319', '318', '320']
index_mat, index_not_mat = [indexes.index(i) for i in new_mat], [indexes.index(i) for i in new_not_mat]
all_indexes = index_mat + index_not_mat
coefs_group = np.zeros((3,8,2,len(fr_bands),len(chan1)))
results_group_l = np.zeros((3,8,2))
#all_indexes_table = mat + not_mat
#all_indexes = np.array(all_indexes)
index_mat, index_not_mat = np.array(index_mat), np.array(index_not_mat)
tprs_f, aucs_f = [], []
fpr_f, tpr_f = [], []

for ind in range(3):
    tprs = []
    aucs = []
    fprl = []
    tprl = []
    for count, (s_ind_1, s_ind_2) in enumerate(zip(index_mat, index_not_mat)):
        A =  np.concatenate(([np.stack([subj_list_features[j] for j in index_mat[index_mat!=s_ind_1]][i][ind],axis = 1)
                          for i in range(index_mat[index_mat!=s_ind_1].shape[0])]),axis=0)
        B =  np.concatenate(([np.stack([subj_list_features[j] for j in index_not_mat[index_not_mat!=s_ind_2]][i][ind],axis = 1)
                          for i in range(index_not_mat[index_not_mat!=s_ind_2].shape[0])]),axis=0)
        C1 = np.stack(subj_list_features[s_ind_1][ind], axis = 1)
        C2 = np.stack(subj_list_features[s_ind_2][ind], axis = 1)
        x_train = np.concatenate((A,B),axis=0)
        y_train = [0]*A.shape[0] + [1]*B.shape[0]
        x_test = np.concatenate((C1, C2),axis=0)
        y_test = [0]*C1.shape[0] + [1]*C2.shape[0]
        x_train = x_train.reshape(x_train.shape[0],-1)
        x_test = x_test.reshape(x_test.shape[0],-1)
        # test size 0.2
        x_train, _, y_train, _ = train_test_split(x_train, y_train, test_size=0.1)
        ac, auc, model, predictions = predict(x_train, x_test, y_train, y_test)
        fpr, tpr, _ = roc_curve(y_test, predictions[:,0], pos_label=0)
        fprl.append(predictions)
        tprl.append(y_test)
        roc_auc = metrics.auc(fpr, tpr)
        interp_tpr = np.interp(mean_fpr, fpr, tpr)
        interp_tpr[0] = 0.0
        tprs.append(interp_tpr)
        aucs.append(roc_auc)
        for name, i in zip(['patterns_', 'filters_'],[0,1]):
            coef = get_coef(model, name, inverse_transform=True)
            coefs_group[ind,count,i,...] = coef.reshape(len(fr_bands),-1)
        results_group_l[ind,count, 0] = ac
        results_group_l[ind,count, 1] = auc
    tpr_f.append(tprl)
    fpr_f.append(fprl)
    tprs_f.append(tprs)
    aucs_f.append(aucs)


In [ ]:
df_1 = pd.DataFrame(results_group_l[0,...], columns=['ac','auc'])
df_2 = pd.DataFrame(results_group_l[1,...], columns=['ac','auc'])
df_3 = pd.DataFrame(results_group_l[2,...], columns=['ac','auc'])
writer = pd.ExcelWriter(os.path.join('/Users/ilyamikheev/Downloads/', 'suc_not_suc_classification_tasks_linear.xlsx'), engine='xlsxwriter')
df_1.to_excel(writer, sheet_name='241')
df_2.to_excel(writer, sheet_name='242')
df_3.to_excel(writer, sheet_name='244')

writer.save()

In [ ]:
df_1 = pd.DataFrame(results_group_t[0,...], columns=['ac','auc'])
df_2 = pd.DataFrame(results_group_t[1,...], columns=['ac','auc'])
df_3 = pd.DataFrame(results_group_t[2,...], columns=['ac','auc'])
writer = pd.ExcelWriter(os.path.join('/Users/ilyamikheev/Downloads/', 'suc_not_suc_classification_tasks_lgb.xlsx'), engine='xlsxwriter')
df_1.to_excel(writer, sheet_name='241')
df_2.to_excel(writer, sheet_name='242')
df_3.to_excel(writer, sheet_name='244')

writer.save()

In [ ]:
import matplotlib.pyplot as plt

from sklearn import svm
from sklearn.metrics import auc
from sklearn.metrics import RocCurveDisplay

random_state = np.random.RandomState(0)

# Run classifier with cross-validation and plot ROC curves

tprs = tprs_f[2]
aucs = aucs_f[2]
fig, ax = plt.subplots(figsize=(12, 8))
for i, (tpr, fpr) in enumerate(zip(tpr_f[2], fpr_f[2])):
    viz = RocCurveDisplay.from_predictions(
        y_true=tpr,
        y_pred=fpr[:,0].tolist(),
        name=f"ROC fold {i+1}",
        pos_label = 0,
        ax=ax
    )
ax.plot([0, 1], [0, 1], linestyle="--", lw=2, color="r", label="Chance", alpha=0.8)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
ax.plot(
    mean_fpr,
    mean_tpr,
    color="b",
    label=r"Mean ROC (AUC = %0.2f $\pm$ %0.2f)" % (mean_auc, std_auc),
    lw=2,
    alpha=0.8,
)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
ax.fill_between(
    mean_fpr,
    tprs_lower,
    tprs_upper,
    color="grey",
    alpha=0.2,
    label=r"$\pm$ 1 std. dev.",
)

ax.set(
    xlim=[-0.05, 1.05],
    ylim=[-0.05, 1.05],
    title="Receiver operating characteristic Logistic reression",
)
ax.legend(loc="lower right")
plt.show()

In [ ]:
fig.savefig(os.path.join('/Users/ilyamikheev/Downloads/', 'lgb_suc_not_suc_tasks.png'), format='png', bbox_inches='tight', dpi=600)

In [ ]:
from itertools import compress
a_l = list(compress(im[1], results_group_t[1][:,1]>0.5))
fin_ar = np.mean(a_lf, axis=0).reshape((7,95))

In [ ]:
fin_ar = np.mean(a_l, axis=0).reshape(len(fr_bands),len(chan1))

In [ ]:
# for linear model
from itertools import compress
a_l = list(compress(coefs_group[2], results_group_l[2][:,1]>0.5))

In [ ]:
a_l[0].shape

In [ ]:
from sklearn.preprocessing import minmax_scale

minmax_scale(a_l[0].reshape(1,-1), feature_range=(-1,1), axis=1)

In [ ]:
a_lf = []
s = a_l[0].shape
for i in range(len(a_l)):
    k = minmax_scale(a_l[i].reshape(2,-1), feature_range=(-1,1), axis=1)
    a_lf.append(k.reshape(s))
fin_ar = np.mean(a_lf, axis=0)

In [ ]:
# for imp
fin_ar = np.mean(a_lf, axis=0).reshape((7,95))

In [ ]:
# for lm
fin_ar = np.mean(a_lf, axis=0)

In [ ]:
fin_ar.min()

In [ ]:
def plot_patterns(ar):
    #vmin = - np.amax(ar)
    #vmax = np.amax(ar)
    vmax = 1
    vmin = -1
    fig, axes = plt.subplots(nrows=2, ncols=7, figsize=(30, 20))
    for name, pos, plot_name, ind in zip(('patterns_', 'filters_'),(0.82,0.5),
                                         ('Patterns','Filters'),(0,1)):
        for i,key in enumerate(list(fr_bands.keys())):
            a = mne.viz.plot_topomap(ar[ind,i,:],info,vmin=vmin,vmax=vmax, axes=axes[ind,i],
                                 show = False)
            axes[ind,i].set_title(label='{}-{} Hz'.format(*fr_bands[key]), fontdict = {'fontsize': 40, 'fontweight' : 'semibold'})
            mne.viz.tight_layout()
        plt.figtext(0.5,pos,'{}'.format(plot_name), va="center", ha="center", size=44, fontweight = 'semibold')
    m = cm.ScalarMappable(cmap='RdBu_r')
    m.set_array([vmin, vmax])
    cax = fig.add_axes([1, 0.3, 0.03, 0.38])
    cb = fig.colorbar(m, cax)
    cb.ax.tick_params(labelsize=40)
    return fig

In [ ]:
fig = plot_patterns(fin_ar)

In [ ]:
fig.savefig(os.path.join('/Users/ilyamikheev/Downloads/', 'feature_patterns_244_group.png'), format='png', bbox_inches='tight', dpi=600)

In [ ]:
def plot_topo(ar, info):
    #vmin = np.amin(ar)
    vmin = 0
    vmax = np.amax(ar)
    fig, axes = plt.subplots(nrows=1, ncols=7, figsize=(30, 20))
    ss = 0
    for i, key in enumerate(list(fr_bands.keys())):
        if i in range(7):
            evoked = mne.EvokedArray(ar[i,:].reshape(-1,1),
                             info, tmin=0.)
            a = evoked.plot_topomap(axes=axes[ss], time_format=None, colorbar = False,  times=[0], cmap='hot',
                                    size = 5, show_names = False, vmin = 0, show = False)
            axes[ss].set_title(label = '{}-{} Hz'.format(*fr_bands[key]), fontdict = {'fontsize': 40, 'fontweight' : 'semibold'})
            mne.viz.tight_layout()
            ss+=1
    m = cm.ScalarMappable(cmap='hot')
    m.set_array([vmin, vmax])
    cax = fig.add_axes([1, 0.3, 0.03, 0.38])
    cb = fig.colorbar(m, cax)
    cb.ax.tick_params(labelsize=40)
    plt.figtext(0.5, 0.7, '{}'.format('Features importance'), va="center", ha="center", size=60, fontweight = 'semibold')
    return fig

In [ ]:
info = epochs_list[0].info
info1 = mne.create_info(info.ch_names, ch_types= 'eeg',
                        sfreq = 250)
info1.set_montage(montage)

In [ ]:
from scipy.spatial.qhull import Delaunay

In [ ]:
from matplotlib import cm
fig = plot_topo(fin_ar, info1)

In [ ]:
fig.savefig(os.path.join('/Users/ilyamikheev/Downloads/', 'feature_imp_242_hot.png'), format='png', bbox_inches='tight', dpi=600)
#plt.save(os.path.join(path_subj_topo, 'cluster_test_244_fin_3.png'), format='png', dpi=600)
plt.close(fig)

In [ ]:
len(index_not_mat)

In [ ]:
# classification using all subjects in both groups and all subjects linear
index_mat_n, index_not_mat_n = index_mat, index_not_mat
results_all, coefs_all = [], []
list_of_len = [len(indexes), len(index_mat), len(index_not_mat)]

tprs_f, aucs_f = [], []
fpr_f, tpr_f = [], []
for group_ind, subj_type in enumerate([[*range(len(indexes))],index_mat,index_not_mat]):
    subj_type = np.array(subj_type)
    results_sgroup = np.zeros((list_of_len[group_ind], 6))
    coefs_group_s = np.zeros((len(dict_cls),list_of_len[group_ind],2,len(fr_bands),len(chan1)))
    for i, key in enumerate(list(dict_cls.keys())):
        ind = dict_cls[key]
        tprs = []
        aucs = []
        fprl = []
        tprl = []
        for count, p in enumerate(subj_type): 
            A = np.concatenate(([np.stack([subj_list_features[j] for j in subj_type[subj_type!=p]][it][ind[0]],axis = 1) 
                                for it in range((subj_type.shape[0])-1)]),axis=0)
            B = np.concatenate(([np.stack([subj_list_features[j] for j in subj_type[subj_type!=p]][it][ind[1]],axis = 1) 
                                for it in range(subj_type.shape[0]-1)]),axis=0)
            C1 = np.stack(subj_list_features[p][ind[0]],axis = 1) 
            C2 = np.stack(subj_list_features[p][ind[1]],axis = 1)
            x_train = np.concatenate((A,B),axis=0)
            y_train = [0]*A.shape[0] + [1]*B.shape[0] 
            x_test = np.concatenate((C1, C2), axis=0)
            y_test = [0]*C1.shape[0] + [1]*C2.shape[0]  
            x_train = x_train.reshape(x_train.shape[0],-1)
            x_test = x_test.reshape(x_test.shape[0],-1)
            x_train, _, y_train, _ = train_test_split(x_train, y_train, test_size=0.1)
            ac, auc, model, predictions = predict(x_train, x_test, y_train, y_test)
            fpr, tpr, _ = roc_curve(y_test, predictions[:,0], pos_label=0)
            fprl.append(predictions)
            tprl.append(y_test)
            roc_auc = metrics.auc(fpr, tpr)
            interp_tpr = np.interp(mean_fpr, fpr, tpr)
            interp_tpr[0] = 0.0
            tprs.append(interp_tpr)
            aucs.append(roc_auc)
            for name, i1 in zip(['patterns_', 'filters_'],[0,1]):
                coef = get_coef(model, name, inverse_transform=True)
                coefs_group_s[i,count,i1,...] = coef.reshape(len(fr_bands),-1)
            results_sgroup[count, i] = ac
            results_sgroup[count, i+3] = auc
        tpr_f.append(tprl)
        fpr_f.append(fprl)
        tprs_f.append(tprs)
        aucs_f.append(aucs)
    coefs_all.append(coefs_group_s)
    results_all.append(results_sgroup)

In [ ]:
results_all[2].shape

In [ ]:
index_not_mat

In [ ]:
#df_1 = pd.DataFrame(results_all[0], columns=list_names_subj, index=indexes)
#df_2 = pd.DataFrame(results_all[1], columns=list_names_subj, index=index_mat)
#df_3 = pd.DataFrame(results_all[2], columns=list_names_subj, index=index_not_mat)
writer = pd.ExcelWriter(os.path.join('/Users/ilyamikheev/Downloads/', 'cross_subjects_classification_tasks_linear.xlsx'), engine='xlsxwriter')
df_1.to_excel(writer, sheet_name='all_subj')
df_2.to_excel(writer, sheet_name='mat')
df_3.to_excel(writer, sheet_name='not_mat')

writer.save()

In [ ]:
np.concatenate(results_all,axis=0).reshape(3,6)

In [ ]:
list_names_subj

In [ ]:
datafr_groups = pd.DataFrame(results_all, columns=list_names_subj, index=['all_sibjects','mathematicians', 'not_mathematicians'])

In [ ]:
datafr_groups

In [ ]:
random_state = np.random.RandomState(0)
from sklearn.metrics import auc
#tprs = list(compress(tprs_f[0], results_all[0][:,3]>0.5))
tprs = tprs_f[8]
#aucs = list(compress(aucs_f[0], results_all[0][:,3]>0.5))
aucs = aucs_f[8]

fig, ax = plt.subplots(figsize=(12, 8))

ax.plot([0, 1], [0, 1], linestyle="--", lw=2, color="r", label="Chance", alpha=0.8)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
ax.plot(
    mean_fpr,
    mean_tpr,
    color="b",
    label=r"Mean ROC (AUC = %0.2f $\pm$ %0.2f)" % (mean_auc, std_auc),
    lw=2,
    alpha=0.8,
)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
ax.fill_between(
    mean_fpr,
    tprs_lower,
    tprs_upper,
    color="grey",
    alpha=0.2,
    label=r"$\pm$ 1 std. dev.",
)

ax.set(
    xlim=[-0.05, 1.05],
    ylim=[-0.05, 1.05],
    title="Receiver operating characteristic Logistic reression",
)
ax.legend(loc="lower right")
plt.show()

In [ ]:
fig.savefig(os.path.join('/Users/ilyamikheev/Downloads/', 'lr_roc_auc_241_242_not_mat_cl.png'), format='png', bbox_inches='tight', dpi=600)

In [ ]:
a_lf = []
a_l = coefs_all[2][2, results_all[2][:,5]>0.5,...]
s = a_l[0].shape
for i in range(len(a_l)):
    k = minmax_scale(a_l[i].reshape(2,-1), feature_range=(-1,1), axis=1)
    a_lf.append(k.reshape(s))
fin_ar = np.mean(a_lf, axis=0)

In [ ]:
fig = plot_patterns(fin_ar)

In [ ]:
fig.savefig(os.path.join('/Users/ilyamikheev/Downloads/', 'feature_patterns_241_242_not_math_cl.png'), format='png', bbox_inches='tight', dpi=600)

In [ ]:
# classification using all subjects in both groups and all subjects lgb
index_mat_n, index_not_mat_n = index_mat, index_not_mat
results_all, coefs_all = [], []
list_of_len = [len(indexes), len(index_mat), len(index_not_mat)]

tprs_f, aucs_f = [], []
fpr_f, tpr_f = [], []
im = []
for group_ind, subj_type in enumerate([[*range(len(indexes))],index_mat,index_not_mat]):
    subj_type = np.array(subj_type)
    results_sgroup = np.zeros((list_of_len[group_ind], 6))
    coefs_group_s = np.zeros((len(dict_cls),list_of_len[group_ind],2,len(fr_bands),len(chan1)))
    for i, key in enumerate(list(dict_cls.keys())):
        im1 = []
        ind = dict_cls[key]
        tprs = []
        aucs = []
        fprl = []
        tprl = []
        for count, p in enumerate(subj_type):
            A = np.concatenate(([np.stack([subj_list_features[j] for j in subj_type[subj_type!=p]][it][ind[0]],axis = 1)
                                for it in range((subj_type.shape[0])-1)]),axis=0)
            B = np.concatenate(([np.stack([subj_list_features[j] for j in subj_type[subj_type!=p]][it][ind[1]],axis = 1)
                                for it in range(subj_type.shape[0]-1)]),axis=0)
            C1 = np.stack(subj_list_features[p][ind[0]],axis = 1)
            C2 = np.stack(subj_list_features[p][ind[1]],axis = 1)
            x_train = np.concatenate((A,B),axis=0)
            y_train = [0]*A.shape[0] + [1]*B.shape[0]
            x_test = np.concatenate((C1, C2), axis=0)
            y_test = [0]*C1.shape[0] + [1]*C2.shape[0]
            x_train = x_train.reshape(x_train.shape[0],-1)
            x_test = x_test.reshape(x_test.shape[0],-1)
            x_train, _, y_train, _ = train_test_split(x_train, y_train, test_size=0.1)
            sc = StandardScaler()
            sc.fit(x_train)
            x_train = sc.transform(x_train)
            x_test = sc.transform(x_test)
            lgb = LGBMClassifier(objective='binary')
            lgb.fit(x_train, y_train)
            feature_importances = (lgb.feature_importances_ / sum(lgb.feature_importances_)) * 100
            im1.append(feature_importances)
            predictions = lgb.predict_proba(x_test)
            auc = roc_auc_score(y_test, predictions[:,1])
            ac = balanced_accuracy_score(y_test, lgb.predict(x_test))
            fpr, tpr, _ = roc_curve(y_test, predictions[:,0], pos_label=0)
            fprl.append(predictions)
            tprl.append(y_test)
            roc_auc = metrics.auc(fpr, tpr)
            interp_tpr = np.interp(mean_fpr, fpr, tpr)
            interp_tpr[0] = 0.0
            tprs.append(interp_tpr)
            aucs.append(roc_auc)
            results_sgroup[count, i] = ac
            results_sgroup[count, i+3] = auc
        tpr_f.append(tprl)
        fpr_f.append(fprl)
        tprs_f.append(tprs)
        aucs_f.append(aucs)
        im.append(im1)
    results_all.append(results_sgroup)

In [ ]:
df_1 = pd.DataFrame(results_all[0], columns=list_names_subj, index=indexes)
df_2 = pd.DataFrame(results_all[1], columns=list_names_subj, index=index_mat)
df_3 = pd.DataFrame(results_all[2], columns=list_names_subj, index=index_not_mat)
writer = pd.ExcelWriter(os.path.join('/Users/ilyamikheev/Downloads/', 'cross_subjects_classification_tasks_lgb.xlsx'), engine='xlsxwriter')
df_1.to_excel(writer, sheet_name='all_subj')
df_2.to_excel(writer, sheet_name='mat')
df_3.to_excel(writer, sheet_name='not_mat')

writer.save()

In [ ]:
random_state = np.random.RandomState(0)
from sklearn.metrics import auc
#tprs = list(compress(tprs_f[0], results_all[0][:,3]>0.5))
tprs = tprs_f[8]
#aucs = list(compress(aucs_f[0], results_all[0][:,3]>0.5))
aucs = aucs_f[8]

fig, ax = plt.subplots(figsize=(12, 8))

ax.plot([0, 1], [0, 1], linestyle="--", lw=2, color="r", label="Chance", alpha=0.8)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
ax.plot(
    mean_fpr,
    mean_tpr,
    color="b",
    label=r"Mean ROC (AUC = %0.2f $\pm$ %0.2f)" % (mean_auc, std_auc),
    lw=2,
    alpha=0.8,
)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
ax.fill_between(
    mean_fpr,
    tprs_lower,
    tprs_upper,
    color="grey",
    alpha=0.2,
    label=r"$\pm$ 1 std. dev.",
)

ax.set(
    xlim=[-0.05, 1.05],
    ylim=[-0.05, 1.05],
    title="Receiver operating characteristic LightGBM",
)
ax.legend(loc="lower right")
plt.show()

In [ ]:
len(im)

In [ ]:
fig.savefig(os.path.join('/Users/ilyamikheev/Downloads/', 'lgb_roc_auc_241_242_not_math_cl.png'), format='png', bbox_inches='tight', dpi=600)

In [ ]:
a_l = list(compress(im[8], results_all[1][:,5]>0.5))
fin_ar = np.mean(a_l, axis=0).reshape((7,95))

In [ ]:
fig = plot_topo(fin_ar, info1)

In [ ]:
fig.savefig(os.path.join('/Users/ilyamikheev/Downloads/', 'feature_imp_241_242_not_mat_cl_hot.png'), format='png', bbox_inches='tight', dpi=600)
#plt.save(os.path.join(path_subj_topo, 'cluster_test_244_fin_3.png'), format='png', dpi=600)
plt.close(fig)